# Model comparisons

In [1]:
import cv2
from face_alignment import FaceAlignment, LandmarksType
import os
import urllib.request as urlreq

from loreal_poc.dataloaders.loaders import DataLoader300W
from loreal_poc.dataloaders.wrappers import CroppedDataLoader

from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NMEMean
from loreal_poc.tests.base import Test, TestDiff
from loreal_poc.marks.facial_parts import FacialParts

In [2]:
dl = DataLoader300W(dir_path="300W/sample")
chosen_idx = 1
image, ground_truth_landmarks, _ = dl[chosen_idx]

In [3]:
facealignment_model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False))
opencv_model = OpenCVWrapper()

In [4]:
# from dataclasses import fields
def create_report(models, dataloader, tests, facial_parts):
    results = []
    for model in models:
        # for __facial_part in fields(FacialParts):
        #  _facial_part = __facial_part.default
        for facial_part in facial_parts:
            for test_cls in tests:
                test_result = test_cls(metric=NMEMean, threshold=1).run(
                    model=model,
                    dataloader=CroppedDataLoader(dataloader, part=facial_part),
                    dataloader_ref=dl,
                    facial_part=facial_part,
                )
                results.append(test_result.to_dict())
    return results


report = create_report(
    [opencv_model, facealignment_model],
    dl,
    [TestDiff],
    [
        FacialParts.BOTTOM_HALF.value,
        FacialParts.UPPER_HALF.value,
        FacialParts.LEFT_HALF.value,
        FacialParts.RIGHT_HALF.value,
    ],
)

/Users/bazire/Work/loreal-poc/loreal_poc/tests/performance.py:47: RuntimeWarning: Mean of empty slice
  mes = np.nanmean(es, axis=1)
/Users/bazire/Work/loreal-poc/loreal_poc/tests/performance.py:85: RuntimeWarning: Mean of empty slice
  return np.nanmean(NMEs.get(prediction_result, marks))
/Users/bazire/Work/loreal-poc/.venv/lib/python3.11/site-packages/face_alignment/api.py:147: UserWarning: No faces were detected.
  warnings.warn("No faces were detected.")
/Users/bazire/Work/loreal-poc/loreal_poc/tests/performance.py:47: RuntimeWarning: Mean of empty slice
  mes = np.nanmean(es, axis=1)


In [5]:
import pandas as pd

final_report = pd.DataFrame(report).groupby(["facial_part", "model_name", "metric_value"]).describe()

In [6]:
final_report

threshold                               \
                                           count mean std  min  25%  50%  75%   
facial_part model_name    metric_value                                          
bottom half FaceAlignment 0.000082           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
left half   FaceAlignment 0.656253           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
            OpenCV        0.644057           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
right half  FaceAlignment 0.341428           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
            OpenCV        0.390821           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
upper half  FaceAlignment 0.223175           1.0  1.0 NaN  1.0  1.0  1.0  1.0   
            OpenCV        0.040216           1.0  1.0 NaN  1.0  1.0  1.0  1.0   

                                             
                                        max  
facial_part model_name    metric_value       
bottom half FaceAlignment 0.000082      1.0  
left half   FaceAlignment 0.656253      1.0  
            OpenCV        0.644057      1.0  
right half  FaceAlignment 0.341428      1.0  
            OpenCV        0.390821      1.0  
upper half  FaceAlignment 0.223175      1.0  
            OpenCV        0.040216      1.0